# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [65]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [66]:
# Import API key
from api_keys import g_key

file = "output_data/cities.csv" 
weather = pd.read_csv(file) 

weather = pd.read_csv(file) 
weather.set_index('City Name', inplace=True) 
weather.drop(columns=['Unnamed: 0'], inplace=True) 
weather.reset_index()



,City Name,Date,Long,Lat,Temp,Humid,Wind,clouds,Country
0,Vaini,1612023597,-175.2000,-21.2000,73.40,94,4.61,90,TO
1,Pangnirtung,1612023197,-65.7125,66.1451,-9.40,77,2.26,1,CA
2,Yellowknife,1612023218,-114.3525,62.4560,-18.99,76,5.75,20,CA
3,Albany,1612023161,-73.9662,42.6001,14.00,53,3.00,0,US
4,Ankang,1612023598,109.0172,32.6800,49.06,48,0.85,99,CN
...,...,...,...,...,...,...,...,...,...
554,Itupiranga,1612023458,-49.3267,-5.1347,89.60,66,4.61,40,BR
555,Morondava,1612023462,44.2833,-20.2833,83.08,74,5.84,1,MG
556,Omboué,1612023404,9.2618,-1.5746,81.63,76,10.47,44,GA
557,Sitka,1612023393,-135.3300,57.0531,37.40,60,9.22,75,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [67]:
locations = weather[["Lat", "Long"]].astype(float)
weight = weather['Humid']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2.5)

fig.add_layer(heat_layer)


In [50]:
hotel_df = weather[weather.eval('Temp>=65 & Temp < 90 & (Humid < 40) & clouds <= 50')]
hotel_df.reset_index(inplace=True)
hotel_df


,City Name,Date,Long,Lat,Temp,Humid,Wind,clouds,Country
0,Atar,1611786577,-13.0499,20.5169,72.43,18,10.36,0,MR
1,Inhapim,1611786906,-42.1200,-19.5492,86.00,39,4.61,0,BR
2,Tessalit,1611786911,1.0114,20.1986,65.39,22,8.23,0,ML
3,Tigre,1611786613,-58.5796,-34.4260,82.40,36,12.66,0,AR
4,Yaan,1611787186,8.5667,7.3833,80.13,16,1.83,35,NG
5,Adrar,1611787186,-10.0711,20.5022,67.01,23,9.62,0,MR
6,Moussoro,1611786928,16.4901,13.6408,70.27,16,10.04,4,TD
7,Raymondville,1611786941,-97.7831,26.4815,75.20,19,10.36,1,US
8,Diffa,1611786946,12.6113,13.3154,70.16,18,9.60,0,NE
9,Valparaíso,1611786717,-71.6273,-33.0393,73.40,38,16.11,0,CL


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in ideal.iterrows():
    lat = row['Lat']
    long = row['Long']
    city = row['City Name']
    #append the lat and long from each row and add to 
    params["location"] = f'{lat},{long}'

    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, "Hotel"] = results[0]['name']
    except :
        pass
    time.sleep(1)
    print("------------")

    hotel_df.dropna(how='any', inplace=True)
    hotel_df

Retrieving Results for Index 0: Atar.
------------
Retrieving Results for Index 1: Inhapim.
------------
Retrieving Results for Index 3: Tigre.
------------
Retrieving Results for Index 7: Raymondville.
------------
Retrieving Results for Index 8: Diffa.
------------
Retrieving Results for Index 9: Valparaíso.
------------
Retrieving Results for Index 10: Bako.
------------


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [86]:
gmaps.configure(api_key=g_key)

In [90]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


figure_layout = {
    'width': '450px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…